# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [14]:
#https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import re
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer


from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /Users/gimoonnam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gimoonnam/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/gimoonnam/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gimoonnam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('DisasterResponseTable',engine)
X = df.message.values
Y = df.iloc[:,4:]
X[1:10]

array(['Is the Hurricane over or is it not over',
       'Looking for someone but no name',
       'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
       'says: west side of Haiti, rest of the country today and tonight',
       'Information about the National Palace-',
       'Storm at sacred heart of jesus',
       'Please, we need tents and water. We are in Silo, Thank you!',
       'I would like to receive the messages, thank you',
       'I am in Croix-des-Bouquets. We have health issues. They ( workers ) are in Santo 15. ( an area in Croix-des-Bouquets )'],
      dtype=object)

In [16]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
stop_words = stopwords.words("english")
        
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok, pos='v').lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [17]:
# class LemmaTokenizer:
#     def __init__(self):
#         self.wnl = WordNetLemmatizer()
#     def __call__(self, text):
#         tokens = [self.wnl.lemmatize(t, pos='v') for t in word_tokenize(text)]
#         return tuple(tokens)
    
# lem = LemmaTokenizer()

# print(lem('This was the first document.'))

# corpus = [
#     'This was the first document.',
#     'This document is the second document.',
#     'And this went the third one.',
#     'Is this the first document?',
# ]

# vectorizer = CountVectorizer(tokenizer=tokenize)
# X = vectorizer.fit_transform(corpus[0:2])

# print(vectorizer.get_feature_names_out())
# print(X.toarray(), X.shape)

In [18]:
# tfidf_trans = TfidfTransformer()
# print(tfidf_trans.fit_transform(X).toarray())

# print(tfidf_trans.idf_)

In [19]:
# tfidf_vec = TfidfVectorizer(tokenizer=tokenize) 

# print(tfidf_vec.fit_transform(corpus).toarray())

# print(tfidf_vec.get_feature_names_out())


In [20]:
# tfidf_vec.idf_

In [21]:
# pipeline = Pipeline([
#         ('vect', CountVectorizer(tokenizer=tokenize)),
#         ('tfidf', TfidfTransformer())
# ])
# pipeline.fit_transform(corpus).toarray()

In [22]:
def display_results(y_test, y_pred):
    #labels = np.unique(y_pred)
    #confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    #print("Labels:", labels)
    #print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [23]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 100)))
    ])

In [24]:
xgb_pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(XGBClassifier(objective='binary:logistic')))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [25]:

X_train, X_test, y_train, y_test = train_test_split(X, Y)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [26]:
%%time

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

# display results
display_results(y_test, y_pred)

Accuracy: related                   0.799139
request                   0.891194
offer                     0.996773
aid_related               0.776241
medical_help              0.920701
medical_products          0.948210
search_and_rescue         0.973721
security                  0.979714
military                  0.967881
water                     0.944368
food                      0.923467
shelter                   0.927770
clothing                  0.983249
money                     0.972491
missing_people            0.990164
refugees                  0.961272
death                     0.956662
other_aid                 0.862609
infrastructure_related    0.937145
transport                 0.955279
buildings                 0.955279
electricity               0.976333
tools                     0.994314
hospitals                 0.990164
shops                     0.996158
aid_centers               0.989089
other_infrastructure      0.956355
weather_related           0.869986
floods    

In [27]:
cl_report_df = pd.DataFrame.from_dict(classification_report(y_test.values, y_pred, target_names=Y.columns.values, output_dict=True)).T

display(cl_report_df)


/Users/gimoonnam/Gits/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/gimoonnam/Gits/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/gimoonnam/Gits/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
related,0.802876,0.974037,0.880213,4930.0
request,0.892857,0.421847,0.572979,1126.0
offer,0.000000,0.000000,0.000000,21.0
aid_related,0.810216,0.621942,0.703704,2780.0
medical_help,0.675676,0.047259,0.088339,529.0
medical_products,0.727273,0.046110,0.086721,347.0
search_and_rescue,0.928571,0.071038,0.131980,183.0
security,0.333333,0.007634,0.014925,131.0
military,0.555556,0.023810,0.045662,210.0
water,0.864865,0.216704,0.346570,443.0


In [28]:
%%time

# train classifier
xgb_pipeline.fit(X_train, y_train)

# predict on test data
y_pred_xgb = xgb_pipeline.predict(X_test)

# display results
display_results(y_test, y_pred_xgb)

/Users/gimoonnam/Gits/.venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Accuracy: related                   0.816198
request                   0.903181
offer                     0.996773
aid_related               0.769940
medical_help              0.929000
medical_products          0.955740
search_and_rescue         0.972645
security                  0.979714
military                  0.970493
water                     0.960811
food                      0.950515
shelter                   0.946673
clothing                  0.987859
money                     0.975872
missing_people            0.990933
refugees                  0.965422
death                     0.968649
other_aid                 0.867835
infrastructure_related    0.935915
transport                 0.958045
buildings                 0.964807
electricity               0.978638
tools                     0.994160
hospitals                 0.990011
shops                     0.996158
aid_centers               0.989550
other_infrastructure      0.953281
weather_related           0.885662
floods    

In [29]:
cl_report_xgb_df = pd.DataFrame.from_dict(classification_report(y_test.values, y_pred_xgb, target_names=Y.columns.values, output_dict=True)).T

display(cl_report_xgb_df)


/Users/gimoonnam/Gits/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/gimoonnam/Gits/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/gimoonnam/Gits/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
related,0.832561,0.948073,0.886571,4930.0
request,0.794537,0.594139,0.679878,1126.0
offer,0.000000,0.000000,0.000000,21.0
aid_related,0.777106,0.647122,0.706183,2780.0
medical_help,0.648889,0.275992,0.387268,529.0
medical_products,0.709220,0.288184,0.409836,347.0
search_and_rescue,0.543860,0.169399,0.258333,183.0
security,0.400000,0.015267,0.029412,131.0
military,0.602273,0.252381,0.355705,210.0
water,0.755435,0.627540,0.685573,443.0


From the results, two things can be inferred, there is something wrong with **related** column and **child_alone** column.

In [30]:
# investigate "related" and "child-alone" column
Y["related"].value_counts()

related
1    19906
0     6122
Name: count, dtype: int64

In [31]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

Few importatnt words like **water, blocked road, medical supplies** are used during a disaster response. So we can create a Custom Transformers like **StartingNounExtractor**, **StartingVerbExtractor** and, **LengthExtractor** and add them to our pipeline.


In [32]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)


In [33]:
# https://www.guru99.com/pos-tagging-chunking-nltk.html

class StartingNounExtractor(BaseEstimator, TransformerMixin):

    def starting_noun(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['NN', 'NNS', 'NNP', 'NNPS'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_noun)
        return pd.DataFrame(X_tagged)

In [34]:
# Not useful in this case
class LengthExtractor(BaseEstimator, TransformerMixin): 
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return  pd.Series(X).apply(len).values.reshape(-1,1)

### Using FeatureUnion

In [35]:
rand_pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            #('length', LengthExtractor()),
            #('starting_noun', StartingNounExtractor()),
            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 100)))
    ])

In [36]:

boost_pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            #('length', LengthExtractor()),
            ('starting_noun', StartingNounExtractor()),
            ('starting_verb', StartingVerbExtractor())
        ])),

        ('xgbclassifier', MultiOutputClassifier(XGBClassifier(objective='binary:logistic',random_state = 42)))
    ])

In [37]:
%%time

# train classifier
rand_pipeline.fit(X_train, y_train)

#predict on test data
y_pred_rand = rand_pipeline.predict(X_test)

#display results
display_results(y_test, y_pred_rand)

/Users/gimoonnam/Gits/.venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
%%time

# train classifier
boost_pipeline.fit(X_train, y_train)

#predict on test data
y_pred_boost = boost_pipeline.predict(X_test)

#display results
display_results(y_test, y_pred_boost)

Accuracy: related                   0.822499
request                   0.901491
offer                     0.994314
aid_related               0.778393
medical_help              0.923313
medical_products          0.956047
search_and_rescue         0.971569
security                  0.982019
military                  0.972491
water                     0.965729
food                      0.952666
shelter                   0.948210
clothing                  0.988781
money                     0.976641
missing_people            0.988935
refugees                  0.967573
death                     0.970954
other_aid                 0.870140
infrastructure_related    0.933764
transport                 0.956969
buildings                 0.958506
electricity               0.982327
tools                     0.994467
hospitals                 0.987859
shops                     0.994621
aid_centers               0.988474
other_infrastructure      0.954972
weather_related           0.878746
floods    

As we can see adding Custom Transformers like **StartingNounExtractor**, **StartingVerbExtractor** to our pipeline, improves the accuracy. Also, XGBoost classifier workes better than random forest. So we'll apply GridsearchCV on XGBoost.
**LengthExtractor** degrades accuracy.


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
#REF : https://xgboost.readthedocs.io/en/latest/python/python_api.html
# https://www.kaggle.com/tilii7/hyperparameter-grid-search-with-xgboost
parameters = {
#         'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#         'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
#         'features__text_pipeline__vect__max_features': (None, 5000, 10000),
#         'features__text_pipeline__tfidf__use_idf': (True, False),
#         'xgbclassifier__estimator__n_estimators': [50, 1000],
        'xgbclassifier__estimator__learning_rate': [0.1, 0.5],
#         'xgbclassifier__estimator__max_depth': [3,5],
#         'xgbclassifier__estimator__gamma': [0.5, 2, 5],
#         'features__transformer_weights': (
#             {'text_pipeline': 1, 'starting_verb': 0.5,'starting_noun': 0.5},
#             {'text_pipeline': 0.5, 'starting_verb': 1,'starting_noun': 0.5},
#             {'text_pipeline': 1, 'starting_verb': 0.5,'starting_noun': 1},
#             {'text_pipeline': 0.8, 'starting_verb': 1,'starting_noun': 0.5},
#         )
    }

cv = GridSearchCV(boost_pipeline, param_grid=parameters,cv = 5)


In [ ]:
%%time
cv.fit(X_train, y_train)

In [ ]:
# predict on test data
y_pred_final = cv.predict(X_test)

# display results
display_results(y_test, y_pred_final)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
print(classification_report(y_test.values, y_pred_final, target_names=Y.columns.values))

### 9. Export your model as a pickle file

In [ ]:
import pickle

pickle.dump(cv, open('models/classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.